In [2]:
import sagemaker
from sagemaker.huggingface import (
    HuggingFaceModel    
)

In [15]:
role = sagemaker.get_execution_role()
hub = {
    'HF_MODEL_ID':'veronica320/QA-for-Event-Extraction',
    'HF_TASK':'question-answering'}

huggingface_model = HuggingFaceModel(
    env=hub,                                                
    role=role,                                              
    transformers_version="4.6",                             # Transformers version used
    pytorch_version="1.7",                                  # PyTorch version used
    py_version='py36',                                      # Python version used
)




In [16]:
encoder = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.t2.large",
    endpoint_name="bertqa"
)

------!

In [37]:
out1 = encoder.predict({"context": "PARENT-TEACHER CONFERENCE REMINDER - Springfield Elementary School Chicago Date: 11/25/2023. To the parent(s) of __Isabella__________________________________,Just a reminder that we have a parent-teacher conference scheduled for ________Robert Connor Family______________on _11/25/2023_____________________ at _____10:30______________ a.m./p.m.in room number __225__________. Our conference will run approximately ___30____ minutes long.Please contact the school t (____240___)__5891234___________________ if you are unable to make this scheduled appointment. I am looking forward to meeting with you and discussing your child’s progress.Your child’s teacher", "question": "when is the meeting"})

out2 = encoder.predict({"context": "PARENT-TEACHER CONFERENCE REMINDER - Springfield Elementary School Chicago Date: 11/25/2023. To the parent(s) of __Isabella__________________________________,Just a reminder that we have a parent-teacher conference scheduled for ________Robert Connor Family______________on _11/25/2023_____________________ at _____10:30______________ a.m./p.m.in room number __225__________. Our conference will run approximately ___30____ minutes long.Please contact the school t (____240___)__5891234___________________ if you are unable to make this scheduled appointment. I am looking forward to meeting with you and discussing your child’s progress.Your child’s teacher", "question": "where is the meeting"})

In [38]:
print (out1)
print (out2)

{'score': 0.38873326778411865, 'start': 275, 'end': 285, 'answer': '11/25/2023'}
{'score': 0.6068448424339294, 'start': 37, 'end': 74, 'answer': 'Springfield Elementary School Chicago'}


In [25]:
out1 = encoder.predict({"context": "PARENT-TEACHER CONFERENCE REMINDER - Springfield Elementary School Chicago Date: 11/25/2023. To the parent(s) of __Isabella__________________________________,Just a reminder that we have a parent-teacher conference scheduled for ________Robert Connor Family______________on _11/25/2023_____________________ at _____10:30______________ a.m./p.m.in room number __225__________. Our conference will run approximately ___30____ minutes long.Please contact the school t (____240___)__5891234___________________ if you are unable to make this scheduled appointment. I am looking forward to meeting with you and discussing your child’s progress.Your child’s teacher", "question": "is there a meeting in the given context"})
print (out1)

In [26]:
print (out1)

{'score': 0.00013135570043232292, 'start': 37, 'end': 66, 'answer': 'Springfield Elementary School'}


In [28]:
out1 = encoder.predict({"context": "Hi how are you doing. Good to know you", "question": "when is the meeting"})
print (out1)

{'score': 0.004302245564758778, 'start': 0, 'end': 21, 'answer': 'Hi how are you doing.'}


In [89]:
import boto3
import json

client = boto3.client('sagemaker-runtime')

endpoint_name = "bertner"                                       # Your endpoint name.
content_type = "application/json"                                     # The MIME type of the input data in the request body.
accept = "application/json"   
input_data =  {"inputs":{"question": "is there a meeting in the given context","context": "PARENT-TEACHER CONFERENCE REMINDER - Springfield Elementary School Chicago Date: 11/25/2023. To the parent(s) of __Isabella__________________________________,Just a reminder that we have a parent-teacher conference scheduled for ________Robert Connor Family______________on _11/25/2023_____________________ at _____10:30______________ a.m./p.m.in room number __225__________. Our conference will run approximately ___30____ minutes long.Please contact the school t (____240___)__5891234___________________ if you are unable to make this scheduled appointment. I am looking forward to meeting with you and discussing your childs progress.Your child’s teacher"}}
    # The desired MIME type of the inference in the response.
payload =json.dumps(input_data)
print(payload)
str(payload)
out1 = encoder.predict(input_data)
print (out1)
type(payload)
#type(input_data)


{"inputs": {"question": "is there a meeting in the given context", "context": "PARENT-TEACHER CONFERENCE REMINDER - Springfield Elementary School Chicago Date: 11/25/2023. To the parent(s) of __Isabella__________________________________,Just a reminder that we have a parent-teacher conference scheduled for ________Robert Connor Family______________on _11/25/2023_____________________ at _____10:30______________ a.m./p.m.in room number __225__________. Our conference will run approximately ___30____ minutes long.Please contact the school t (____240___)__5891234___________________ if you are unable to make this scheduled appointment. I am looking forward to meeting with you and discussing your childs progress.Your child\u2019s teacher"}}
{'score': 0.00031935962033458054, 'start': 37, 'end': 66, 'answer': 'Springfield Elementary School'}


In [ ]:
response = client.invoke_endpoint(
    EndpointName=endpoint_name, 
    CustomAttributes="", 
    ContentType=content_type,
    Accept=accept,
    Body=payload
    )

In [84]:
str(encoder)

'<sagemaker.huggingface.model.HuggingFacePredictor object at 0x7f2ebc99b610>'

In [92]:
import sagemaker, boto3, json
from sagemaker.session import Session

sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()

from sagemaker import image_uris, model_uris, script_uris
from sagemaker.model import Model
from sagemaker.predictor import Predictor
from sagemaker.utils import name_from_base

# model_version="*" fetches the latest version of the model.
infer_model_id, infer_model_version = "huggingface-eqa-distilbert-base-uncased", "*"

endpoint_name = name_from_base(f"insight-bert")

inference_instance_type = "ml.p2.xlarge"

# Retrieve the inference docker container uri.
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    image_scope="inference",
    model_id=infer_model_id,
    model_version=infer_model_version,
    instance_type=inference_instance_type,
)
# Retrieve the inference script uri.
deploy_source_uri = script_uris.retrieve(
    model_id=infer_model_id, model_version=infer_model_version, script_scope="inference"
)
# Retrieve the base model uri.
base_model_uri = model_uris.retrieve(
    model_id=infer_model_id, model_version=infer_model_version, model_scope="inference"
)
# Create the SageMaker model instance. Note that we need to pass Predictor class when we deploy model through Model class,
# for being able to run inference through the sagemaker API.
model = Model(
    image_uri=deploy_image_uri,
    source_dir=deploy_source_uri,
    model_data=base_model_uri,
    entry_point="inference.py",
    role=aws_role,
    predictor_cls=Predictor,
    name=endpoint_name,
)
# deploy the Model.
base_model_predictor = model.deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    endpoint_name=endpoint_name,
)

---------!

In [114]:
import boto3
import json

client = boto3.client('sagemaker-runtime')

endpoint_name = "insight-bert-2023-11-13-20-40-49-689"                                       # Your endpoint name.
content_type = "application/json"                                     # The MIME type of the input data in the request body.
accept = "application/json"   
input_data =  {"inputs":{"question": "is there a meeting in the given context","context": "PARENT-TEACHER CONFERENCE REMINDER - Springfield Elementary School Chicago Date: 11/25/2023. To the parent(s) of __Isabella__________________________________,Just a reminder that we have a parent-teacher conference scheduled for ________Robert Connor Family______________on _11/25/2023_____________________ at _____10:30______________ a.m./p.m.in room number __225__________. Our conference will run approximately ___30____ minutes long.Please contact the school t (____240___)__5891234___________________ if you are unable to make this scheduled appointment. I am looking forward to meeting with you and discussing your childs progress.Your child’s teacher"}}
    # The desired MIME type of the inference in the response.
input_data =    [
    "Where is in the meeting ?",
    "PARENT-TEACHER CONFERENCE REMINDER - Springfield Elementary School Chicago Date: 11/25/2023. To the parent(s) of __Isabella__________________________________,Just a reminder that we have a parent-teacher conference scheduled for ________Robert Connor Family______________on _11/25/2023_____________________ at _____10:30______________ a.m./p.m.in room number __225__________. Our conference will run approximately ___30____ minutes long.Please contact the school t (____240___)__5891234___________________ if you are unable to make this scheduled appointment. I am looking forward to meeting with you and discussing your childs progress.Your child’s teacher"
    ,
]
payload =json.dumps(input_data).encode("utf-8")
print(payload)
str(payload)
out1 = base_model_predictor.predict(payload, {"ContentType": "application/list-text", "Accept": "application/json;verbose"})
print (out1)
type(payload)



b'["Where is in the meeting ?", "PARENT-TEACHER CONFERENCE REMINDER - Springfield Elementary School Chicago Date: 11/25/2023. To the parent(s) of __Isabella__________________________________,Just a reminder that we have a parent-teacher conference scheduled for ________Robert Connor Family______________on _11/25/2023_____________________ at _____10:30______________ a.m./p.m.in room number __225__________. Our conference will run approximately ___30____ minutes long.Please contact the school t (____240___)__5891234___________________ if you are unable to make this scheduled appointment. I am looking forward to meeting with you and discussing your childs progress.Your child\\u2019s teacher"]'
b'{"score": 0.10199644416570663, "start": 37, "end": 66, "answer": "Springfield Elementary School"}'


bytes

In [122]:
response = client.invoke_endpoint(
    EndpointName=endpoint_name, 
    CustomAttributes="", 
    ContentType='application/list-text',
    Accept=accept,
    Body=payload
    )

In [123]:
print(response)

{'ResponseMetadata': {'RequestId': '5373cce8-6011-42fd-8175-e8c67fa2d0fe', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '5373cce8-6011-42fd-8175-e8c67fa2d0fe', 'x-amzn-invoked-production-variant': 'AllTraffic', 'date': 'Mon, 13 Nov 2023 21:00:20 GMT', 'content-type': 'application/json', 'content-length': '97', 'connection': 'keep-alive'}, 'RetryAttempts': 0}, 'ContentType': 'application/json', 'InvokedProductionVariant': 'AllTraffic', 'Body': <botocore.response.StreamingBody object at 0x7f2ebf58dd50>}


In [124]:
newline, bold, unbold = "\n", "\033[1m", "\033[0m"
def parse_response(query_response):
    model_predictions = json.loads(query_response)
    answer = (model_predictions["answer"],)
    return answer
#answer = parse_response(response)
#print(response['Body'].read())
answer = json.loads(response['Body'].read().decode())
print( 
        f"model answer: {bold}{answer}{unbold}{newline}"
    )

model answer: {'score': 0.10199644416570663, 'start': 37, 'end': 66, 'answer': 'Springfield Elementary School'}

